In [1]:
#importing libs
import pandas as pd
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [11]:
#reading the dataset
data_train = pd.read_csv('D:\\AIML\\IIT Sasthra\\-Covid-19-cases\\dataset\\train_data_covid.csv')
data_test = pd.read_csv('D:\\AIML\\IIT Sasthra\\-Covid-19-cases\\dataset\\test_data_covid.csv')

In [12]:
data_train


,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed,OxygenTanks,PopulationDensityPerSqKm
0,1,2020-01-30,6:00 PM,Kerala,1,0,0,0,1,8262,867
1,2,2020-01-31,6:00 PM,Kerala,1,0,0,0,1,9136,61
2,3,2020-02-01,6:00 PM,Kerala,2,0,0,0,2,3826,316
3,4,2020-02-02,6:00 PM,Kerala,3,0,0,0,3,9789,806
4,5,2020-02-03,6:00 PM,Kerala,3,0,0,0,3,882,369
...,...,...,...,...,...,...,...,...,...,...,...
14996,14997,2021-05-17,8:00 AM,Lakshadweep,-,-,3633,14,4767,4046,490
14997,14998,2021-05-17,8:00 AM,Madhya Pradesh,-,-,629741,6992,731385,1195,595
14998,14999,2021-05-17,8:00 AM,Maharashtra,-,-,4826371,81486,5378452,9289,195
14999,15000,2021-05-17,8:00 AM,Manipur,-,-,32674,578,39729,576,296


In [13]:
#taking the required features
data_train_abs = data_train[["Date", "State/UnionTerritory", "Cured", "Deaths", "Confirmed"]]
data_test_abs = data_test[["Date", "State/UnionTerritory", "Cured", "Confirmed"]]

In [14]:
data_train_abs

,Date,State/UnionTerritory,Cured,Deaths,Confirmed
0,2020-01-30,Kerala,0,0,1
1,2020-01-31,Kerala,0,0,1
2,2020-02-01,Kerala,0,0,2
3,2020-02-02,Kerala,0,0,3
4,2020-02-03,Kerala,0,0,3
...,...,...,...,...,...
14996,2021-05-17,Lakshadweep,3633,14,4767
14997,2021-05-17,Madhya Pradesh,629741,6992,731385
14998,2021-05-17,Maharashtra,4826371,81486,5378452
14999,2021-05-17,Manipur,32674,578,39729


In [15]:
#columns for outlier removal
columns_to_clean = ["Cured", "Confirmed", "Deaths"]

In [16]:
#removing outliers from training dataset
def remove_outliers_iqr(data, columns):
    
    data_cleaned = data.copy()

    for column in columns:
        q1 = data_cleaned[column].quantile(0.25)
        q3 = data_cleaned[column].quantile(0.75)

        IQR = q3 - q1
        upper_limit = q3 + 1.5 * IQR
        lower_limit = q1 - 1.5 * IQR

        data_cleaned = data_cleaned[((data_cleaned[column] >= lower_limit) & (data_cleaned[column] <= upper_limit))]

    return data_cleaned

In [17]:
#removing outliers
data_train_cleaned = remove_outliers_iqr(data_train_abs, columns_to_clean)
print(data_train_cleaned)

             Date State/UnionTerritory   Cured  Deaths  Confirmed
0      2020-01-30               Kerala       0       0          1
1      2020-01-31               Kerala       0       0          1
2      2020-02-01               Kerala       0       0          2
3      2020-02-02               Kerala       0       0          3
4      2020-02-03               Kerala       0       0          3
...           ...                  ...     ...     ...        ...
14987  2021-05-17                  Goa  105505    2099     135856
14995  2021-05-17               Ladakh   14741     165      16452
14996  2021-05-17          Lakshadweep    3633      14       4767
14999  2021-05-17              Manipur   32674     578      39729
15000  2021-05-17            Meghalaya   18478     320      23332

[11016 rows x 5 columns]


In [18]:
#number of unique states in dataset
number_of_states_training = len(data_train_cleaned['State/UnionTerritory'].unique())
print("Number of States & UTs in data (train) =", number_of_states_training)
print("Name of States/UTs in Train =", data_train_cleaned['State/UnionTerritory'].unique())

Number of States & UTs in data (train) = 39
Name of States/UTs in Train = ['Kerala' 'Telengana' 'Delhi' 'Rajasthan' 'Uttar Pradesh' 'Haryana'
 'Ladakh' 'Tamil Nadu' 'Karnataka' 'Maharashtra' 'Punjab'
 'Jammu and Kashmir' 'Andhra Pradesh' 'Uttarakhand' 'Odisha' 'Puducherry'
 'West Bengal' 'Chhattisgarh' 'Chandigarh' 'Gujarat' 'Himachal Pradesh'
 'Madhya Pradesh' 'Bihar' 'Manipur' 'Mizoram'
 'Andaman and Nicobar Islands' 'Goa' 'Unassigned' 'Assam' 'Jharkhand'
 'Arunachal Pradesh' 'Tripura' 'Nagaland' 'Meghalaya'
 'Dadra and Nagar Haveli and Daman and Diu'
 'Cases being reassigned to states' 'Sikkim' 'Daman & Diu' 'Lakshadweep']


In [19]:
#total number of days the case has been repeated
state_count = {}

for num in data_train_cleaned["State/UnionTerritory"]:
    if num in state_count:
        state_count[num] += 1
    else:
        state_count[num] = 1

for element, count in state_count.items():
    print(f"Number of occurrences of {element}: {count}")

Number of occurrences of Kerala: 258
Number of occurrences of Telengana: 355
Number of occurrences of Delhi: 115
Number of occurrences of Rajasthan: 273
Number of occurrences of Uttar Pradesh: 164
Number of occurrences of Haryana: 268
Number of occurrences of Ladakh: 437
Number of occurrences of Tamil Nadu: 133
Number of occurrences of Karnataka: 145
Number of occurrences of Maharashtra: 85
Number of occurrences of Punjab: 189
Number of occurrences of Jammu and Kashmir: 419
Number of occurrences of Andhra Pradesh: 155
Number of occurrences of Uttarakhand: 410
Number of occurrences of Odisha: 236
Number of occurrences of Puducherry: 425
Number of occurrences of West Bengal: 150
Number of occurrences of Chhattisgarh: 231
Number of occurrences of Chandigarh: 425
Number of occurrences of Gujarat: 129
Number of occurrences of Himachal Pradesh: 422
Number of occurrences of Madhya Pradesh: 194
Number of occurrences of Bihar: 390
Number of occurrences of Manipur: 420
Number of occurrences of M

In [20]:
#storing each and every states data seperatly
unique_states = data_train_cleaned["State/UnionTerritory"].unique()

for states in unique_states:
    var_name = states.lower().replace(' ', '_')
    globals()[f"data_train_{var_name}"] = data_train_cleaned[data_train_cleaned["State/UnionTerritory"] == states]

In [21]:
#value assigning for train data dynamically
xtrain = []
ytrain = []

for state in unique_states:
    var_name = state.lower().replace(' ', '_')
    state_data = data_train_abs[data_train_abs["State/UnionTerritory"] == state]

    #list of features to drop
    features_to_drop_x = ['Deaths', 'Date'] 
    features_to_drop_y = ['Date', 'State/UnionTerritory', 'Cured', 'Confirmed']
    
    x_train_state = state_data.drop(features_to_drop_x, axis=1)
    y_train_state = state_data.drop(features_to_drop_y, axis=1)

    #assign x train and y train to dynamically created variables
    globals()[f"xtrain_{var_name}"] = x_train_state
    globals()[f"ytrain_{var_name}"] = y_train_state

    #append x train and y train to the list
    xtrain.append(f"xtrain_{var_name}")
    ytrain.append(f"ytrain_{var_name}")

In [22]:
label_encoder = LabelEncoder()

In [23]:
for state_name in xtrain:
    state = globals()[state_name]
    state['State/UnionTerritory'] = label_encoder.fit_transform(state['State/UnionTerritory'])
    globals()[state_name] = state

In [24]:
#model creation
model = Sequential()
model.add(LSTM(units=50, activation='relu', return_sequences=True, input_shape=(1, 3)))
model.add(LSTM(units=50, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

In [25]:
#model compilation
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="mse")

In [26]:
# reshaping the xtrain dataset
for state_name in xtrain:
    state = globals()[state_name]
    state = state.values.reshape((state.shape[0], 1, state.shape[1]))
    globals()[state_name] = state


In [27]:
"""x_train = xtrain_andaman_and_nicobar_islands.values.reshape((xtrain_andaman_and_nicobar_islands.shape[0], 1, xtrain_andaman_and_nicobar_islands.shape[1]))
print(x_train.shape)"""

'x_train = xtrain_andaman_and_nicobar_islands.values.reshape((xtrain_andaman_and_nicobar_islands.shape[0], 1, xtrain_andaman_and_nicobar_islands.shape[1]))\nprint(x_train.shape)'

In [28]:
for state_name in xtrain:
    print(state)

[[[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0    0]]

 [[   0    0   14]]

 [[   0    0   21]]

 [[   0    0   30]]

 [[   0    0   48]]

 [[   0    0   50]]

 [[   0    0   57]]

 [[   0    0   59]]

 [[   0    0   67]]

 [[   0    0 

In [29]:
model.fit(xtrain_kerala, ytrain_kerala, epochs=100)

Epoch 1/100
15/15 [==============================] - 4s 3ms/step - loss: 6465307.0000
Epoch 2/100
15/15 [==============================] - 0s 3ms/step - loss: 6465306.5000
Epoch 3/100
15/15 [==============================] - 0s 3ms/step - loss: 6465305.5000
Epoch 4/100
15/15 [==============================] - 0s 3ms/step - loss: 6465379.5000
Epoch 5/100
15/15 [==============================] - 0s 3ms/step - loss: 6465306.5000
Epoch 6/100
15/15 [==============================] - 0s 3ms/step - loss: 6465325.5000
Epoch 7/100
15/15 [==============================] - 0s 3ms/step - loss: 6465947.5000
Epoch 8/100
15/15 [==============================] - 0s 3ms/step - loss: 6465306.5000
Epoch 9/100
15/15 [==============================] - 0s 3ms/step - loss: 6465307.0000
Epoch 10/100
15/15 [==============================] - 0s 3ms/step - loss: 6465618.0000
Epoch 11/100
15/15 [==============================] - 0s 3ms/step - loss: 6465306.0000
Epoch 12/100
15/15 [==============================] 

In [31]:
# Iterate over each state's training data
for xtrain_state_name, ytrain_state_name in zip(xtrain, ytrain):
    # Get the actual NumPy arrays for the state
    xtrain_state = globals()[xtrain_state_name]
    ytrain_state = globals()[ytrain_state_name]

    # Ensure the shapes are correct
    print("Input shape:", xtrain_state.shape)
    print("Output shape:", ytrain_state.shape)

    # Fit the model for the current state
    model.fit(xtrain_state, ytrain_state, epochs=10, validation_data=(xtrain_state, ytrain_state))


Input shape: (474, 1, 3)
Output shape: (474, 1)
Epoch 1/10
15/15 [==============================] - 1s 53ms/step - loss: 6468645.5000 - val_loss: 6468646.0000
Epoch 2/10
15/15 [==============================] - 0s 7ms/step - loss: 6468646.0000 - val_loss: 6468646.0000
Epoch 3/10
15/15 [==============================] - 0s 7ms/step - loss: 6468645.5000 - val_loss: 6468646.0000
Epoch 4/10
15/15 [==============================] - 0s 7ms/step - loss: 6468646.0000 - val_loss: 6468646.0000
Epoch 5/10
15/15 [==============================] - 0s 7ms/step - loss: 6468646.0000 - val_loss: 6468646.0000
Epoch 6/10
15/15 [==============================] - 0s 7ms/step - loss: 6468646.0000 - val_loss: 6468646.0000
Epoch 7/10
15/15 [==============================] - 0s 7ms/step - loss: 6468646.0000 - val_loss: 6468646.0000
Epoch 8/10
15/15 [==============================] - 0s 7ms/step - loss: 6468646.5000 - val_loss: 6468646.0000
Epoch 9/10
15/15 [==============================] - 0s 7ms/step - loss: